## 공간 데이터 분석

### 데이터 로드

In [24]:
import pandas as pd
import geopandas as gpd

In [36]:
npark_boundary = gpd.read_file('./data/Protected_areas_OECM_Republic_of_Korea_ver_2023.shp', encoding='cp949')
park_data = pd.read_csv('./data/240301_final_data_ver2.csv')
safety_place = pd.read_excel('./data//북한산 안전쉼터 현황.xlsx')
sign_place = pd.read_excel('./data/북한산 다목적 위치표지판 현황.xlsx')

In [26]:
gdf_park_data = gpd.GeoDataFrame(park_data, 
                               geometry=gpd.points_from_xy(park_data.경도_변환, park_data.위도_변환),
                               crs='epsg:4326'
                              )

In [40]:
gdf_safety_place = gpd.GeoDataFrame(safety_place, 
                               geometry=gpd.points_from_xy(safety_place.경도, safety_place.위도),
                               crs='epsg:4326'
                              )

gdf_sign_place = gpd.GeoDataFrame(sign_place, 
                               geometry=gpd.points_from_xy(sign_place.경도, sign_place.위도),
                               crs='epsg:4326'
                              )

In [48]:
safety_place.head(1)

,사무소명,코드명,쉼터명,위 치,위도,경도
0,북한,북한-01,영취천,"N 37°37'3.4"" E 126°59'5.2""",37.617611,126.984778


### 국립공원 경계에서 북한산만 추출하기

In [27]:
def find_boundary(npark_boundary,npark_name):
    npark_boundary = npark_boundary[npark_boundary['DESIG']=='국립공원']
    seoul_npark_boundary = npark_boundary[npark_boundary['ORIG_NAME']==npark_name]
    return seoul_npark_boundary

seoul_npark_boundary = find_boundary(npark_boundary,'북한산')

### 북한산 국립공원 경계랑 사고현황 데이터랑 공간조인(북한산 국립공원 경계안에 있는 사고현황 데이터만 추출)

In [28]:
def sjoin(gdf_park_data,npark_boundary,npark_name):
    gdf_seoul_park_data = gdf_park_data[gdf_park_data['국립공원명']==npark_name]
    seoul_accident = gpd.sjoin(gdf_seoul_park_data, npark_boundary)
    return seoul_accident

seoul_accident = sjoin(gdf_park_data,seoul_npark_boundary,'북한산')

### 지도 시각화

In [29]:
def find_center_latitudes_longtitudes(accident):
    latitudes = accident['위도_변환'].tolist()
    longitudes = accident['경도_변환'].tolist()
    map_center_lat = sum(latitudes) / len(latitudes)
    map_center_lon = sum(longitudes) / len(longitudes)
    return map_center_lat,map_center_lon

map_center_lat,map_center_lon = find_center_latitudes_longtitudes(seoul_accident)

### 유형별 안전사고 분석

In [53]:
import folium
from folium.plugins import HeatMap
import json
from IPython.display import display


vworld_key="BF677CB9-D1EA-3831-B328-084A9AE3CDCC" # VWorld API key
layer = "Satellite" # VWorld layer
tileType = "jpeg" # tile type
accident_list = seoul_accident['사고원인그룹'].value_counts().index

# 사고 원인별 색상 사전 정의
color_dict = {
    '실족추락': 'red',
    '무리한 활동': 'green',
    '길잃음': 'blue',
    '건강문제 및 개인질환': 'yellow',
    '동물 및 해충': 'gray',
    '부주의': 'purple',
    '자연 및 환경': 'skyblue',
}

map_center_lat,map_center_lon = find_center_latitudes_longtitudes(seoul_accident)

tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"
# 기본 지도 객체 생성
m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12,tiles=tiles, attr=attr)

# VWorld Hybrid 타일 추가
folium.TileLayer(
    tiles=f'http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/Hybrid/{{z}}/{{y}}/{{x}}.png',
    attr='VWorld Hybrid',
    name='VWorld Hybrid',
    overlay=True
).add_to(m)

# 사고 원인별로 레이어 그룹 생성 및 추가
for i in accident_list:
    # 사고 원인별 데이터 필터링
    type_accident = seoul_accident[seoul_accident['사고원인그룹'] == i]
    
    # 해당 사고 원인의 색상 가져오기
    accident_color = color_dict.get(i, 'black')  # 사고 원인이 color_dict에 없으면 'black' 사용
    
    # 사고 원인별 레이어 그룹 생성
    layer_group = folium.FeatureGroup(name=i)
    
    # 사고 위치에 대한 CircleMarker 추가
    for idx, row in type_accident.iterrows():
        folium.CircleMarker(
            location=(row['위도_변환'], row['경도_변환']),
            radius=3,
            color=accident_color,
            fill=True,
            fill_color=accident_color,
            fill_opacity=1.0  # 내부 채움 불투명도
        ).add_to(layer_group)
    
    # 레이어 그룹을 지도 객체에 추가
    layer_group.add_to(m)

# seoul_npark_boundary GeoDataFrame을 GeoJson으로 변환 및 추가
geojson_data = json.loads(seoul_npark_boundary.to_json())
folium.GeoJson(
    geojson_data,
    name='geojson',
    style_function=lambda feature: {
        'color': 'yellow',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 지도 표시
display(m)


### 유형별 HeatMap 분석

In [54]:
import folium
from folium.plugins import HeatMap
import json
from IPython.display import display


vworld_key="BF677CB9-D1EA-3831-B328-084A9AE3CDCC" # VWorld API key
layer = "Satellite" # VWorld layer
tileType = "jpeg" # tile type
accident_list = seoul_accident['사고원인그룹'].value_counts().index

# 사고 원인별 색상 사전 정의
color_dict = {
    '실족추락': 'red',
    '무리한 활동': 'green',
    '길잃음': 'blue',
    '건강문제 및 개인질환': 'yello',
    '동물 및 해충': 'gray',
    '부주의': 'purple',
    '자연 및 환경': 'skyblue',
}

map_center_lat,map_center_lon = find_center_latitudes_longtitudes(seoul_accident)

tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"
# 기본 지도 객체 생성
m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12,tiles=tiles, attr=attr)

# VWorld Hybrid 타일 추가
folium.TileLayer(
    tiles=f'http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/Hybrid/{{z}}/{{y}}/{{x}}.png',
    attr='VWorld Hybrid',
    name='VWorld Hybrid',
    overlay=True
).add_to(m)



# 사고 원인별로 레이어 그룹 생성 및 추가
for i in accident_list:
    # 사고 원인별 데이터 필터링
    type_accident = seoul_accident[seoul_accident['사고원인그룹'] == i]
    
    # 사고 위치 데이터 준비 (위도, 경도)
    accident_locations = type_accident[['위도_변환', '경도_변환']].values.tolist()
    
    # 히트맵 레이어 생성
    heat_map = plugins.HeatMap(accident_locations, name=i, radius=15, gradient={0.2: 'blue', 0.4: 'green', 0.6: 'yellow', 0.8: 'orange', 1: 'red'})
    
    # 레이어 그룹 생성 및 히트맵 레이어 추가
    layer_group = folium.FeatureGroup(name=i).add_child(heat_map)
    
    # 레이어 그룹을 지도 객체에 추가
    layer_group.add_to(m)

# seoul_npark_boundary GeoDataFrame을 GeoJson으로 변환 및 추가
geojson_data = json.loads(seoul_npark_boundary.to_json())
folium.GeoJson(
    geojson_data,
    name='geojson',
    style_function=lambda feature: {
        'color': 'yellow',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 지도 표시
display(m)

In [63]:
import folium
from folium.plugins import HeatMap
import json
from IPython.display import display


vworld_key="BF677CB9-D1EA-3831-B328-084A9AE3CDCC" # VWorld API key
layer = "Satellite" # VWorld layer
tileType = "jpeg" # tile type
accident_list = seoul_accident['사고원인그룹'].value_counts().index

# 사고 원인별 색상 사전 정의
color_dict = {
    '실족추락': 'red',
    '무리한 활동': 'green',
    '길잃음': 'blue',
    '건강문제 및 개인질환': 'yello',
    '동물 및 해충': 'gray',
    '부주의': 'purple',
    '자연 및 환경': 'skyblue',
}

map_center_lat,map_center_lon = find_center_latitudes_longtitudes(seoul_accident)

tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"
# 기본 지도 객체 생성
m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12,tiles=tiles, attr=attr)

# VWorld Hybrid 타일 추가
folium.TileLayer(
    tiles=f'http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/Hybrid/{{z}}/{{y}}/{{x}}.png',
    attr='VWorld Hybrid',
    name='VWorld Hybrid',
    overlay=True
).add_to(m)



# 사고 원인별로 레이어 그룹 생성 및 추가
for i in accident_list:
    # 사고 원인별 데이터 필터링
    type_accident = seoul_accident[seoul_accident['사고원인그룹'] == i]
    
    # 사고 위치 데이터 준비 (위도, 경도)
    accident_locations = type_accident[['위도_변환', '경도_변환']].values.tolist()
    
    # 히트맵 레이어 생성
    heat_map = plugins.HeatMap(accident_locations, name=i, radius=15, gradient={0.4: 'green', 0.6: 'yellow', 0.8: 'orange', 1: 'red'})
    
    # 레이어 그룹 생성 및 히트맵 레이어 추가
    layer_group = folium.FeatureGroup(name=i).add_child(heat_map)
    
    # 레이어 그룹을 지도 객체에 추가
    layer_group.add_to(m)

# seoul_npark_boundary GeoDataFrame을 GeoJson으로 변환 및 추가
geojson_data = json.loads(seoul_npark_boundary.to_json())
folium.GeoJson(
    geojson_data,
    name='geojson',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

# # 안전사고표시판 GeoDataFrame을 GeoJson으로 변환
# safety_signs_geojson = json.loads(gdf_sign_place.to_json())

# # folium 지도에 GeoJson 레이어로 안전사고표시판 추가
# folium.GeoJson(
#     safety_signs_geojson,
#     name='Safety Signs',
#     style_function=lambda feature: {
#         'color': 'red',  # 여기서 스타일을 커스텀할 수 있습니다.
#         'weight': 2,
#         'fillOpacity': 0.5
#     },
#     tooltip=folium.GeoJsonTooltip(fields=['위치번호', '위치'], aliases=['위치번호', '위치'])  # 필드명을 실제 GeoDataFrame의 필드명으로 바꿔주세요.
# ).add_to(m)

# 레이어 컨트롤 추가
# folium.LayerControl().add_to(m)


# 안전사고표시판 GeoDataFrame의 각 행에 대해 CircleMarker 추가
for _, row in gdf_sign_place.iterrows():
    # Geometry에서 위도와 경도 추출
    lat, lon = row.geometry.y, row.geometry.x
    # Tooltip으로 표시할 텍스트 생성
    tooltip_text = f"위치 : {row['위치']}, 위치번호: {row['위치번호']}"
    # CircleMarker 추가
    folium.CircleMarker(
        location=[lat, lon],
        radius=3,  # 원의 반지름
        color='blue',  # 원의 테두리 색상
        fill=True,
        weight=0.5,
        fill_color='blue',  # 원을 채우는 색상
        fill_opacity=0.5,  # 채우기 불투명도
        tooltip=tooltip_text  # Tooltip 추가
    ).add_to(m)

# 레이어 컨트롤 추가 (필요한 경우)
folium.LayerControl().add_to(m)

# 안전사고표시판 GeoDataFrame의 각 행에 대해 CircleMarker 추가
for _, row in gdf_safety_place.iterrows():
    # Geometry에서 위도와 경도 추출
    lat, lon = row.geometry.y, row.geometry.x
    # Tooltip으로 표시할 텍스트 생성
    tooltip_text = f"쉼터명 : {row['쉼터명']}, 코드명: {row['코드명']}"
    # CircleMarker 추가
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,  # 원의 반지름
        color='blue',  # 원의 테두리 색상
        weight=0.5,
        fill=True,
        fill_color='blue',  # 원을 채우는 색상
        fill_opacity=0.5,  # 채우기 불투명도
        tooltip=tooltip_text  # Tooltip 추가
    ).add_to(m)

# 레이어 컨트롤 추가 (필요한 경우)
folium.LayerControl().add_to(m)


# 지도 표시
display(m)
